In [ ]:
import math

def gives_angle(perp_distance, eyes_distance):
    hypotenuse = math.sqrt(perp_distance**2 + eyes_distance**2)
    angle = math.atan(perp_distance/eyes_distance)
    return angle

    


In [38]:
import cv2

# Load the Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize the video capture
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    if ret:
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale frame
        blurred_gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # Detect eyes in the blurred grayscale frame
        eyes = eye_cascade.detectMultiScale(blurred_gray, scaleFactor=1.3, minNeighbors=5)

        # Iterate over the detected eyes
        for (ex, ey, ew, eh) in eyes:
            # Extract the eye region from the frame
            eye = frame[ey:ey+eh, ex:ex+ew]

            # Convert the eye region to grayscale
            eye_gray = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)

            # Apply thresholding to obtain a binary image
            _, thresh = cv2.threshold(eye_gray, 30, 255, cv2.THRESH_BINARY)

            # Find contours in the binary image
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Iterate over the contours
            for contour in contours:
                # Calculate the area of the contour
                area = cv2.contourArea(contour)

                # Filter contours based on area to find the dark portion of the eyeball
                if area > 100:
                    # Find the centroid of the contour using the moments
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])

                        # Draw a black dot at the center of the dark portion of the eyeball
                        cv2.circle(eye, (cx, cy), 2, (0, 0, 255), -1)

            # Replace the eye region back in the frame
            frame[ey:ey+eh, ex:ex+ew] = eye

        # Display the frame
        cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()